In [2]:
import findspark
findspark.init('/home/venkat/Downloads/spark-3.2.0-bin-hadoop3.2')

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('mytree').getOrCreate()

In [3]:
from pyspark.ml import Pipeline

In [4]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [5]:
data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

21/11/30 08:13:41 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [6]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [7]:
train_data, test_data = data.randomSplit([0.7,0.3])

In [9]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees = 100)
gbt = GBTClassifier()

In [11]:
dtc_model=dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [12]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [13]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[95,96,97,12...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[127,128,129...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[129,130,131...|   [0.0,40.0]|  [0.0,1.0]|       1.0|
|  0.0|(692,[151,152,153...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [32.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[154,155,156...|   [0.0,40.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[123,124,125...|   [0.0,40.0]|  [0.0,1.0]|       1.0|
|  1.0|(69

In [15]:
gbt_preds.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[95,96,97,12...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[123,124,125...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[124,125,126...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[126,127,128...|[1.22459780736009...|[0.92050260777307...|       0.0|
|  0.0|(692,[127,128,129...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[127,128,129...|[1.18713849010634...|[0.91484464580930...|       0.0|
|  0.0|(692,[128,129,130...|[1.54350200272498...|[0.95635347857270...|       0.0|
|  0.0|(692,[129,130,131...|[-0.7767862799602...|[0.17457087832247...|       1.0|
|  0.0|(692,[151,152,153...|[1.37001072157711...|[0.93934731839670...|       0.0|
|  0.0|(692,[152

In [16]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [17]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [18]:
print('Decision Tree Accuracy')
acc_eval.evaluate(dtc_preds)

Decision Tree Accuracy


0.8888888888888888

In [19]:
print('Random Forest Accuracy')
acc_eval.evaluate(rfc_preds)

Random Forest Accuracy


1.0

In [20]:
print('Gradient Boosted Tree Accuracy')
acc_eval.evaluate(gbt_preds)

Gradient Boosted Tree Accuracy


0.8888888888888888

In [21]:
# Get Feature importance
rfc_model.featureImportances

SparseVector(692, {149: 0.0005, 153: 0.0005, 154: 0.0049, 155: 0.0074, 156: 0.0005, 157: 0.0003, 159: 0.0003, 209: 0.0001, 212: 0.0005, 213: 0.0017, 215: 0.0003, 235: 0.0026, 242: 0.0013, 243: 0.007, 244: 0.0093, 259: 0.0021, 263: 0.0067, 265: 0.0004, 268: 0.0013, 270: 0.0006, 272: 0.014, 273: 0.0091, 274: 0.0021, 287: 0.0017, 289: 0.0066, 290: 0.0073, 294: 0.0007, 296: 0.0034, 298: 0.0017, 299: 0.0004, 301: 0.0075, 315: 0.0007, 317: 0.0133, 319: 0.0017, 320: 0.0009, 322: 0.0006, 331: 0.0006, 341: 0.0014, 349: 0.0005, 350: 0.0123, 351: 0.0245, 352: 0.0162, 353: 0.0005, 354: 0.0033, 355: 0.0011, 356: 0.006, 357: 0.0077, 358: 0.0007, 370: 0.0004, 372: 0.0036, 376: 0.0008, 377: 0.0036, 378: 0.0484, 379: 0.0637, 380: 0.0067, 398: 0.0017, 400: 0.0077, 402: 0.0059, 404: 0.001, 405: 0.0058, 406: 0.0633, 407: 0.0289, 408: 0.0016, 411: 0.0014, 412: 0.0005, 414: 0.0052, 426: 0.0013, 428: 0.0208, 430: 0.0009, 433: 0.0495, 434: 0.0241, 435: 0.0255, 436: 0.001, 438: 0.0021, 440: 0.0084, 443: 0.0053